# Plotting demos
Demos the plotting functions and some related utility

In [1]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir('../')

In [3]:
! pip install cenpy

  Preparing metadata (setup.py) ... done
  Created wheel for cenpy: filename=cenpy-1.0.0.post4-py3-none-any.whl size=29923 sha256=69023446c891e6a6cef3ddaeba46f0b24679d88464b922b2291002d6eb61c92e
  Stored in directory: /home/jovyan/.cache/pip/wheels/94/53/3c/6683b7375d2e8b74e8aa99949599594c014f558bddc6c491b1
Successfully built cenpy


In [4]:
from roc_bike_growth.loader import POI_graph_from_polygon, bike_infra_from_polygon, carall_from_polygon, _fill_edge_geometry, load_roc_proposed
from roc_bike_growth.plotting import add_categorical_legend
from roc_bike_growth.paper_gt import gt_with_existing_full
from roc_bike_growth.graph_utils import combine_nodes, graph_length_km, get_street_segment
from roc_bike_growth.settings import CONFIG

/opt/conda/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
from roc_bike_growth.settings import CONFIG

## Load up everything

In [6]:
rochester = ox.geocode_to_gdf('rochester, ny').geometry[0]

In [7]:
bike_infra = bike_infra_from_polygon(rochester, compose_all=True)

In [8]:
def load_our_proposed(carall: nx.MultiDiGraph = None) -> nx.MultiDiGraph:
    """
    Downloads a cleaned representation of recommendations from the roc-bike-growth
    report.

    Parameters
    -------
    carall: nx.MultiDiGraph
        Rochester carall graph. Can be passed-in to avoid downloading again.
    """

    segments = [  # street, src_intersection, dest_intersection
        ("Monroe Avenue", "culver road", "Howell Street"),
        ("Elmwood Avenue", "Mount Hope Avenue", "South Goodman Street"),
        ("Driving Park Ave", "La Grange ave", "Carthage Drive"),
        ("Avenue E", "Carthage Drive","Saint Paul Street"),
        ("Joseph Avenue", "Cumberland St", "Norton Street"),
        ("Portland Avenue", "North Street", "Central Park"),
        ("State Street", "Andrews Street", "Smith Street"),
        ("Smith Street", "Lake Avenue", "Bausch Street"),
        ("Bausch Street","Suntru Street","Saint Paul Street"),
        ("South Clinton Avenue", "Gregory Street", "East Broad Street"),
        ("Stone Street", "East Broad Street", "East Main Street"),
        ("Saint Paul Street", "East Main Street", "Andrews Street"),

    ]

    # We could probably eliminate the need to download carall here by refactoring get_street_segment.
    # Should probably just refactor this whole thing into a class...
    if carall is None:
        carall = carall_from_polygon(ox.geocode_to_gdf("rochester, ny").geometry[0])
    nodes = []
    for street, src, dest in segments:
        nodes += get_street_segment(carall, street, src, dest)

    proposed = carall.subgraph(set(nodes)).copy()
    
    # drop some weird edges
    streets_to_drop =[
     "South Vincent Street",
     "Bragdon Place",
     ["Linwood Place", "Priem Street"]
    ]
    
    edges_to_drop = []
    for u,v,k,d in proposed.edges(data=True, keys=True):
        if d['name'] in streets_to_drop:
            edges_to_drop.append((u,v,k))
        
    proposed.remove_edges_from(edges_to_drop)
    return proposed

    

In [9]:
carall = carall_from_polygon(rochester, add_pois=True)
pois = nx.get_node_attributes(carall, 'poi').keys()

Exception at Port of Rochester, Lake Ave . This point will be dropped:
 Nominatim could not geocode query "Port of Rochester, Lake Ave  rochester ny"
Exception at 810-846 N GOODMAN ST. This point will be dropped:
 Nominatim could not geocode query "810-846 N GOODMAN ST rochester ny"
Exception at 650-672 E Main St. This point will be dropped:
 Nominatim could not geocode query "650-672 E Main St rochester ny"
Exception at 497-499 S. CLINTON. This point will be dropped:
 Nominatim could not geocode query "497-499 S. CLINTON rochester ny"


In [10]:
proposed = load_our_proposed(carall)

In [11]:
poi_layer = folium.FeatureGroup(name='points of interest',show=True)
for node in pois:
    d = carall.nodes()[node]
    poi_layer.add_child(folium.CircleMarker(
        location=(d['y'],d['x']),
        radius=4,
        **{'color':'#8F2080', 'weight':1, 'fill':True}
    ))

## Compare income adj and no income adj

In [12]:
graph_length_km(proposed)

12.888401000000014

In [15]:
graphs = {
    'existing (122 km)': bike_infra,
    'recommended (13 km)': proposed
}

colors = {
    'existing (122 km)': '#85C0F9',
    'recommended (13 km)':  '#F5793A',
    'points of interest': '#8F2080',
}

m = folium.Map(location=[43.1594283845703, -77.60641012519851], tiles="cartodbpositron", zoom_start=12)
for name, g in graphs.items():
    layer = folium.FeatureGroup(name=name,show=True)
    ox.folium.plot_graph_folium(
    g,
    layer, 
    # popup_attribute='name', 
     **{'color': colors.get(name)})
    layer.add_to(m)
    
poi_layer.add_to(m)

In [16]:
folium.LayerControl(position='bottomright').add_to(m)
add_categorical_legend(m, 'Infrastructure Legend', colors.values(), colors.keys())

In [17]:
m.save('reconnect_roc.html')